In [ ]:
# # dependency installation
# !pip install typing_extensions==4.6.1 pydantic pydantic-core cuml umap-learn pandas numpy matplotlib scikit-learn nltk langdetect bertopic[spacy] openai
# !pip install bertopic flair

In [1]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer
import os

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
import umap
import matplotlib.pyplot as plt
import openai
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic  
from sklearn.metrics import silhouette_score

#find stop words, along with recipe realted stopwords
stop = stopwords.words('english')

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"


/Users/minijain/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Topic Modeling

In [3]:
# Pile all comments into single array

fix = Legallabel
col = LegalPredicted

data = pd.read_csv('final working dataset with only suicide articles.csv')
data = data[data[col] > 0]
comments_all = data['Only Reason']
print("before: ",len(comments_all))
comments_all = comments_all.dropna()
print("before: ",len(comments_all))

before:  4579
before:  4579


In [4]:
len(comments_all)

4579

In [5]:
comments_all = comments_all.to_list()

In [6]:
REDUCE_FREQUENT_WORDS = True
STOP_WORDS_FLAG = True # stops words being used in topics, but not for embeddings
NGRAM_RANGE = (1,2)  
PRETRAINED_SENTENCE_MODEL = 'all-mpnet-base-v2'

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
sentence_model = SentenceTransformer(PRETRAINED_SENTENCE_MODEL)
embeddings = sentence_model.encode(comments_all, show_progress_bar=True)

Batches: 100%|██████████| 144/144 [01:59<00:00,  1.21it/s]


In [9]:
# init model
stopwords = list(stopwords.words('english'))
umap_model = umap.UMAP(n_neighbors=10, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=NGRAM_RANGE)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=REDUCE_FREQUENT_WORDS)
representation_model = KeyBERTInspired()

In [ ]:
#for siz in range(6, 7):

model = BERTopic(top_n_words = 10,
                    n_gram_range = (1,2),
                    language="english",
                    nr_topics=45,
                    calculate_probabilities = True,
                    embedding_model = PRETRAINED_SENTENCE_MODEL,
                    representation_model=representation_model,
                    vectorizer_model=vectorizer_model,
                    umap_model=umap_model)

topics, probs = model.fit_transform(comments_all, embeddings)

cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
embedding1 = model._extract_embeddings(comments_all)  
score = silhouette_score(embedding1, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)
print(" Number of topics: ",len(set(topics))-1)
tit = "All - " + fix
fig = model.visualize_barchart(top_n_topics=100, title = tit)
fig


In [ ]:
# Reduce the number of topics  
model.reduce_topics(comments_all, nr_topics=34)
topics = model.topics_ 
model.visualize_barchart(top_n_topics=100, title = tit)

In [ ]:
# Reduce the number of topics  
model.reduce_topics(comments_all, nr_topics=30)
topics = model.topics_ 
model.visualize_barchart(top_n_topics=100, title = tit)

In [ ]:
model.visualize_topics()

In [ ]:
cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embeddings = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embeddings, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)

In [ ]:
model.visualize_topics()

In [ ]:
# Reduce outliers
#new_topics = model.reduce_outliers(comments_all, topics, probabilities=probs, strategy="probabilities")
#model.update_topics(comments_all, topics=new_topics)

In [ ]:
# model.visualize_heatmap()

In [ ]:
# embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
# model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/safetensors/legal3", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 2 - pytorch
# model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/pytorch/legal3", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 3 - pickle
# model.save("legalbertopic3", serialization="pickle")

In [10]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model = BERTopic.load("legalbertopic3", embedding_model=embedding_model)

In [11]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1376,-1_suicide article_jeffrey epstein_committed s...,NaN,[The article mentions Jeffrey Epstein awaiting...
1,0,1505,0_suicide article_problem article_violation la...,"[suicide article, problem article, violation l...",[The article mentions man killed two people ki...
2,1,949,1_epstein convicted_jeffrey epstein_epstein ki...,"[epstein convicted, jeffrey epstein, epstein k...",[The article mentions Jeffrey Epstein killed j...
3,2,92,2_inflicted gunshot_gunshot wound_helped inmat...,"[inflicted gunshot, gunshot wound, helped inma...",[The article mentions Vicky White jail officia...
4,3,56,3_creator mcafee_john mcafee_mcafee creator_mc...,"[creator mcafee, john mcafee, mcafee creator, ...",[The article mentions John McAfee creator McAf...
5,4,45,4_suicide article_law murder_fatally shot_girl...,"[suicide article, law murder, fatally shot, gi...",[The article mentions Brian Laundrie killed Ga...
6,5,30,5_extradite assange_julian assange_assange fac...,"[extradite assange, julian assange, assange fa...",[The article discusses case Julian Assange fou...
7,6,29,6_suicide article_terrorism charges_committed ...,"[suicide article, terrorism charges, committed...",[The article mentions Matthew L. Perna committ...
8,7,29,7_school shooting_student article_violence sch...,"[school shooting, student article, violence sc...",[The article mentions 17-year-old student Aust...
9,8,28,8_issues allegations_facing allegations_politi...,"[issues allegations, facing allegations, polit...",[The article mentions Eric Greitens former gov...


In [14]:
topics = model.topics_
probs = model.probabilities_

# from existing predictions
left = data['Left'].to_list()
indsleft = []
for i in range(len(comments_all)):
    if left[i] == 1:
        indsleft.append(i)

alltopicsleft = [topics[i] for i in indsleft if i < len(topics)]
allprobsleft = [probs[i] for i in indsleft if i < len(probs)]
allcommentsleft = [comments_all[i] for i in indsleft if i < len(comments_all)]
onetextleft = " ".join(allcommentsleft)

right = data['Right'].to_list()
indsright = []
for i in range(len(comments_all)):
    if right[i] == 1:
        indsright.append(i)

alltopicsright = [topics[i] for i in indsright if i < len(topics)]
allprobsright = [probs[i] for i in indsright if i < len(probs)]
allcommentsright = [comments_all[i] for i in indsright if i < len(comments_all)]
onetextright = " ".join(allcommentsright)

regional = data['Regional'].to_list()
indsreg = []
for i in range(len(comments_all)):
    if regional[i] == 1:
        indsreg.append(i)

alltopicsreg = [topics[i] for i in indsreg if i < len(topics)]
allprobsreg = [probs[i] for i in indsreg if i < len(probs)]
allcommentsreg = [comments_all[i] for i in indsreg if i < len(comments_all)]
onetextreg = " ".join(allcommentsreg)

national = data['National'].to_list()
indsnat = []
for i in range(len(comments_all)):
    if national[i] == 1:
        indsnat.append(i)

alltopicsnat = [topics[i] for i in indsnat if i < len(topics)]
allprobsnat = [probs[i] for i in indsnat if i < len(probs)]
allcommentsnat = [comments_all[i] for i in indsnat if i < len(comments_all)]
onetextnat = " ".join(allcommentsnat)

embeddingsleft = sentence_model.encode(onetextleft, show_progress_bar=True)
embeddingsright = sentence_model.encode(onetextright, show_progress_bar=True)
embeddingsnat = sentence_model.encode(onetextnat, show_progress_bar=True)
embeddingsreg = sentence_model.encode(onetextreg, show_progress_bar=True)


onetopicleft, oneprobleft = model.transform(onetextleft)
onetopicright, oneprobright = model.transform(onetextright)
onetopicreg, oneprobreg = model.transform(onetextreg)
onetopicnat, oneprobnat = model.transform(onetextnat)

topicwiseprobleft = np.array(allprobsleft).mean(axis = 0)
topicwiseprobright = np.array(allprobsright).mean(axis = 0)
topicwiseprobnat = np.array(allprobsnat).mean(axis = 0)
topicwiseprobreg = np.array(allprobsreg).mean(axis = 0)

np.set_printoptions(suppress=True, formatter={'float': '{:0.5f}'.format})
print("Individual averaged: ", np.round(topicwiseprobleft - topicwiseprobright, 5))
print("Combined 1 text: ", np.round(oneprobleft - oneprobright, 5))

print("Individual averaged: ", np.round(topicwiseprobnat - topicwiseprobreg, 5))
print("Combined 1 text: ", np.round(oneprobnat - oneprobreg, 5))

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


Individual averaged:  [-0.01684 0.01961 0.00165 -0.00535 -0.01015 0.00022 -0.00596 0.00230
 0.00368 0.00422 0.00091 0.00874 0.00458 -0.00370 0.00352 0.00264 -0.00008
 -0.00437 -0.00284 -0.00117 0.00198 0.00019 -0.00313 0.00261 -0.00120
 0.00091 0.00082 0.00152 0.00106 0.00377 0.00265 -0.00129 -0.00000 0.00305]
Combined 1 text:  [[0.00000 -0.84858 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
  0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
  0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
  0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000]]
Individual averaged:  [-0.04396 0.03171 -0.00205 -0.00289 0.00453 0.00266 -0.00152 -0.00098
 0.00425 0.00451 0.00389 0.00729 0.00433 -0.00133 0.00145 -0.00034 0.00156
 -0.00113 -0.00295 0.00257 -0.00371 0.00088 0.00155 0.00249 0.00035
 0.00186 0.00004 0.00463 0.00147 0.00248 -0.00111 0.00199 -0.00907
 -0.00034]
Combined 1 text:  [[-0.00000 -0.70868 -0.00000 -0.00000 -0.00000 -0.00000 

In [15]:
df = model.get_topic_info()

topicwiseprobl = np.insert(topicwiseprobleft, 0,-1)
topicwiseprobr = np.insert(topicwiseprobright, 0,-1)
topicwiseprobnat = np.insert(topicwiseprobnat, 0,-1)
topicwiseprobreg = np.insert(topicwiseprobreg, 0,-1)
df['Left - Individual Averaged'] = topicwiseprobl
df['Right - Individual Averaged'] = topicwiseprobr
df['National - Individual Averaged'] = topicwiseprobnat
df['Regional - Individual Averaged'] = topicwiseprobreg

oneprobl = np.insert(oneprobleft, 0,-1)
oneprobr = np.insert(oneprobright, 0,-1)
oneprobn = np.insert(oneprobnat, 0,-1)
oneprobre = np.insert(oneprobreg, 0,-1)
df['Left - One combined text'] = oneprobl
df['Right - One combined text'] = oneprobr
df['National - One combined text'] = oneprobn
df['Regional - One combined text'] = oneprobre

df['Left v/s Right - Individual Averaged'] = topicwiseprobl - topicwiseprobr
df['National v/s Regional - Individual Averaged'] = topicwiseprobnat - topicwiseprobreg
df['Left v/s Right - Combined Text'] = oneprobl - oneprobr
df['National v/s Regional - Combined Text'] = oneprobn - oneprobre

df.to_csv('legal betopic results.csv')

In [16]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1376,-1_suicide article_jeffrey epstein_committed s...,NaN,[The article mentions Jeffrey Epstein awaiting...
1,0,1505,0_suicide article_problem article_violation la...,"[suicide article, problem article, violation l...",[The article mentions man killed two people ki...
2,1,949,1_epstein convicted_jeffrey epstein_epstein ki...,"[epstein convicted, jeffrey epstein, epstein k...",[The article mentions Jeffrey Epstein killed j...
3,2,92,2_inflicted gunshot_gunshot wound_helped inmat...,"[inflicted gunshot, gunshot wound, helped inma...",[The article mentions Vicky White jail officia...
4,3,56,3_creator mcafee_john mcafee_mcafee creator_mc...,"[creator mcafee, john mcafee, mcafee creator, ...",[The article mentions John McAfee creator McAf...
5,4,45,4_suicide article_law murder_fatally shot_girl...,"[suicide article, law murder, fatally shot, gi...",[The article mentions Brian Laundrie killed Ga...
6,5,30,5_extradite assange_julian assange_assange fac...,"[extradite assange, julian assange, assange fa...",[The article discusses case Julian Assange fou...
7,6,29,6_suicide article_terrorism charges_committed ...,"[suicide article, terrorism charges, committed...",[The article mentions Matthew L. Perna committ...
8,7,29,7_school shooting_student article_violence sch...,"[school shooting, student article, violence sc...",[The article mentions 17-year-old student Aust...
9,8,28,8_issues allegations_facing allegations_politi...,"[issues allegations, facing allegations, polit...",[The article mentions Eric Greitens former gov...


In [17]:
ben = data.copy()

In [20]:
probabilities = model.probabilities_
for i in range(probabilities.shape[1]):
    ben['Topic' + str(i)] = probabilities[:, i]


In [22]:
ben.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'Unnamed: 0.7', 'Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'id',
       'title', 'content', 'Publisher', 'Date Published',
       'Financial - Predicted', 'School - Predicted', 'Health - Predicted',
       'Legal - Predicted', 'None - Predicted', 'Reasoning', 'Only Reason',
       'National', 'Regional', 'Left', 'Right', 'Fringe/Questionable',
       'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6',
       'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11', 'Topic12',
       'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17', 'Topic18',
       'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23', 'Topic24',
       'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29', 'Topic30',
       'Topic31', 'Topic32', 'Topic33'],
      dtype='object')

In [23]:
ben = ben.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'Unnamed: 0.7', 'Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4','Reasoning'])

In [24]:
ben = ben.rename(columns={FinancialPredicted: FinancialLabel, LegalPredicted: Legallabel, SchoolPredicted : SchoolLabel, HealthPredicted : HealthLabel})

In [25]:
ben.columns

Index(['id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem',
       'None - Predicted', 'Only Reason', 'National', 'Regional', 'Left',
       'Right', 'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3',
       'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10',
       'Topic11', 'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16',
       'Topic17', 'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22',
       'Topic23', 'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28',
       'Topic29', 'Topic30', 'Topic31', 'Topic32', 'Topic33'],
      dtype='object')

In [27]:
ben.to_csv('Legal problem articles topic model results.csv')

In [ ]:
leftarticles = data[data['Left'] == 1]
rightarticles = data[data['Right'] == 1]

topicsleft, probsleft = model.transform(leftarticles['Only Reason'].to_list())
topicsright, probsright = model.transform(rightarticles['Only Reason'].to_list())

print(np.round(np.array(probsleft).mean(axis = 0) - np.array(probsright).mean(axis = 0), 5))